# Gaussian process

In [10]:
import os
import os.path
import numpy as np
import time
from tqdm import tqdm

assert os.path.exists('data')

In [11]:
def load_dataset(data_source_filename):
  print("Loading %s... " % data_source_filename.split("/")[-1], end="")
  t = time.time()
  dataset = dict()
  with np.load(data_source_filename, allow_pickle=True) as source_file:
    for key in source_file.keys():
      # print(key)
      dataset[key] = source_file[key].tolist()
  print("done (%.1fs)" % (time.time()-t), flush=True)
  return dataset

data_source_filenames = [os.path.join('data', fn) for fn in os.listdir('data')
                            if os.path.isfile(os.path.join('data', fn)) and fn[-3:]=='npz']
data_source_filenames

['data/StClare_facs_danish.npz',
 'data/SDHK_Latin.npz',
 'data/StClare_facs_latin.npz',
 'data/StClare_dipl_danish.npz',
 'data/StClare_dipl_latin.npz',
 'data/SDHK_Swedish.npz',
 'data/Colonia.npz',
 'data/SemEval2015.npz']

In [12]:
#from sklearn.gaussian_process import GaussianProcessRegressor
#from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import accuracy_score
import GPy

bin_width = 25
n_samples_limit = 3000
n_features_ard_limit = 100
n_features_max = 1000

results = dict()

In [13]:
#for data_source_filename in set(data_source_filenames).difference(set(results.keys())):
for data_source_filename in data_source_filenames[1:2]:
    dataset = load_dataset(data_source_filename)
    y = np.asarray([np.mean(item['date']) if type(item['date']) is tuple else item['date'] for item in dataset['data']], np.int)
    #print(" Classes in data: %s" % np.unique(y))
    
    training_set = dataset['folds']['train']
    validation_set = dataset['folds']['val']
    test_set = dataset['folds']['test']
    training_validation_set = list()
    training_validation_set.extend(training_set)
    training_validation_set.extend(validation_set)
    
    # Subsample
    if len(training_validation_set) > n_samples_limit:
        print(" Training data too large (%i), subsampling to %i" % (len(training_validation_set), n_samples_limit))
        training_validation_set = np.asarray(training_validation_set)[np.linspace(0, len(training_validation_set), n_samples_limit, endpoint=False, dtype=np.int)]

    results[data_source_filename] = dict()
    print(" Running estimations", flush=True)
    for feature_set in list(dataset['feature_sets'].keys()):
        X = dataset['feature_sets'][feature_set]
        print("  Loaded feature set %s" % feature_set)
        n_samples, n_features = X.shape
        y_pred = np.zeros(y.shape)

        # Chi^2 feature selection
        if n_features > n_features_max:
            print("   Reducing feature space")
            given_data = list()
            given_data.extend(training_set)
            given_data.extend(validation_set)
            feature_selector = SelectKBest(chi2, k=n_features_max).fit(X[given_data], y[given_data])
            X = feature_selector.transform(X)
            n_features = X.shape[1]

        if n_features < n_features_ard_limit:
            print("   ARD kernel (%i features)" % (n_features))
            kernel = GPy.kern.RBF(input_dim=n_features, ARD=True)
            #kernel = ConstantKernel()*RBF(length_scale=np.ones(n_features)) + WhiteKernel()
        else:
            print("   No ARD kernel (%i features)" % (n_features))
            kernel = GPy.kern.RBF(input_dim=n_features)
            #kernel = ConstantKernel()*RBF() + WhiteKernel()

        #estimator = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True)
        #estimator.fit(X[training_validation_set, :].todense(), y[training_validation_set])
        print("  Making the estimator", flush=True)
        estimator = GPy.models.GPRegression(X[training_validation_set, :].todense(), np.vstack(y[training_validation_set]), kernel)
        estimator.optimize_restarts(num_restarts = 5, messages=False)
        
        y_pred, y_sigma_pred = estimator.predict(np.asarray(X[test_set, :].todense()))
        #y_pred, y_sigma_pred = estimator.predict(X[test_set, :].todense())        
        #y_pred, y_sigma_pred = estimator.predict(X[test_set, :].todense(), return_std=True)
        y_pred_binned = bin_width*(y_pred//bin_width)
        y_true_binned = bin_width*(y[test_set]//bin_width)
        results[data_source_filename][feature_set] = {'accuracy': accuracy_score(y_true_binned, y_pred_binned),
                                                      'y_pred': y_pred_binned, 'y_true': y_true_binned,
                                                      'y__pred_regr': y_pred, 'y_true_regr': y[test_set],
                                                      'y_sigma_regr': y_sigma_pred}

    print()
    print(" Results")
    for feature_set in results[data_source_filename].keys():
        print("  %s, %.1f%%" % (feature_set, 100*results[data_source_filename][feature_set]['accuracy']))
        print("  unique predicted classes: %s" % np.unique(results[data_source_filename][feature_set]['y_pred']))

Loading SDHK_Latin.npz... done (1.6s)
 Training data too large (6058), subsampling to 3000
 Running estimations
  Loaded feature set bow_words
   Reducing feature space
   No ARD kernel (1000 features)
  Making the estimator


 /home/fredrik/anaconda3/lib/python3.7/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /home/fredrik/anaconda3/lib/python3.7/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/fredrik/anaconda3/lib/python3.7/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 1/5, f = 16496.784277690036
Optimization restart 2/5, f = 15518.492757979144
Optimization restart 3/5, f = 15518.492757513788
Optimization restart 4/5, f = 18263.95785741519
Optimization restart 5/5, f = 16336.184595510487
  Loaded feature set tfidf_words
   Reducing feature space
   No ARD kernel (1000 features)
  Making the estimator
Optimization restart 1/5, f = 15544.71417137913
Optimization restart 2/5, f = 16335.634422844563


 /home/fredrik/anaconda3/lib/python3.7/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /home/fredrik/anaconda3/lib/python3.7/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/fredrik/anaconda3/lib/python3.7/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 3/5, f = 22891.841686470267
Optimization restart 4/5, f = 16337.125194360862
Optimization restart 5/5, f = 16336.38644926733
  Loaded feature set character_ngram_1
   No ARD kernel (103 features)
  Making the estimator
Optimization restart 1/5, f = 16335.564325504609
Optimization restart 2/5, f = 16337.022830769194
Optimization restart 3/5, f = 16335.572772053183
Optimization restart 4/5, f = 15399.664429152283
Optimization restart 5/5, f = 15399.66442912534
  Loaded feature set character_ngram_2
   Reducing feature space
   No ARD kernel (1000 features)
  Making the estimator
Optimization restart 1/5, f = 14927.92543260844
Optimization restart 2/5, f = 16335.569191441182
Optimization restart 3/5, f = 14927.925432729784
Optimization restart 4/5, f = 14927.925432596101
Optimization restart 5/5, f = 14927.925432685774
  Loaded feature set character_ngram_3
   Reducing feature space
   No ARD kernel (1000 features)
  Making the estimator
Optimization restart 1/5, f = 

...

Optimization restart 1/5, f = 14915.357935482685
Optimization restart 2/5, f = 16335.568518489754
Optimization restart 3/5, f = 14915.357935480075
Optimization restart 4/5, f = 14915.35793549479
Optimization restart 5/5, f = 16335.570929807274
  Loaded feature set word_ngram_2
   Reducing feature space
   No ARD kernel (1000 features)
  Making the estimator
Optimization restart 1/5, f = 16335.58331051464


 /home/fredrik/anaconda3/lib/python3.7/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 2/5, f = 15820.2687651651
Optimization restart 3/5, f = 15869.768597371627
Optimization restart 4/5, f = 16335.974281600484
Optimization restart 5/5, f = 16335.568639522058
  Loaded feature set word_ngram_3
   Reducing feature space
   No ARD kernel (1000 features)
  Making the estimator
Optimization restart 1/5, f = 16044.102773290568
Optimization restart 2/5, f = 16335.583712175936
Optimization restart 3/5, f = 16335.687197499046
Optimization restart 4/5, f = 16335.777082045812


 /home/fredrik/anaconda3/lib/python3.7/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 5/5, f = 16467.170467337728

 Results
  bow_words, 27.0%
  unique predicted classes: [1075. 1150. 1175. 1200. 1225. 1250. 1275. 1300. 1325. 1350. 1375. 1400.
 1425. 1450. 1475. 1500. 1525. 1550. 1575.]
  tfidf_words, 27.5%
  unique predicted classes: [1150. 1175. 1200. 1225. 1250. 1275. 1300. 1325. 1350. 1375. 1400. 1425.
 1450. 1475. 1500. 1525. 1550.]
  character_ngram_1, 34.2%
  unique predicted classes: [1075. 1125. 1150. 1175. 1200. 1225. 1250. 1275. 1300. 1325. 1350. 1375.
 1400. 1425. 1450. 1475. 1500. 1525. 1600. 1750.]
  character_ngram_2, 36.1%
  unique predicted classes: [1125. 1150. 1175. 1200. 1225. 1250. 1275. 1300. 1325. 1350. 1375. 1400.
 1425. 1450. 1475. 1500. 1525.]
  character_ngram_3, 39.6%
  unique predicted classes: [1100. 1150. 1175. 1200. 1225. 1250. 1275. 1300. 1325. 1350. 1375. 1400.
 1425. 1450. 1475. 1500. 1525.]
  word_ngram_1, 42.1%
  unique predicted classes: [1175. 1200. 1225. 1250. 1275. 1300. 1325. 1350. 1375. 1400. 1425. 1450.
 1

In [14]:
for data_source_filename in results.keys():
    dataset = load_dataset(data_source_filename)
    print(" Saving %s... " % data_source_filename.split("/")[-1], end="")
    t = time.time()
    if 'gaussianprocess' in dataset.keys():
        dataset.pop('gaussianprocess')
    np.savez_compressed(data_source_filename, **dataset, gaussianprocess=results[data_source_filename])
    print("done (%.1fs)" % (time.time()-t), flush=True)

Loading SDHK_Latin.npz... done (1.5s)
 Saving SDHK_Latin.npz... done (6.5s)
